In [1]:
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.linear_model import LassoCV, Lasso, LogisticRegressionCV
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from Project.feature.CMIM import CMIMSelector
from Project.feature.JMI import JMISelector
from Project.feature.FastCMIM import fast_cmim
from matplotlib import pyplot as plt

What to measure?
- accuracy of logistic regression with selected columns for each selection method
    + Binarized data:
        * JMI
        * CMIM
        * Lasso
        * SVM
        * FastCMIM
    + Original data:
        * Lasso
        * SVM


In [2]:
def base_model(X_train, X_test, y_train, y_test, columns_supp, method):
    clf = LogisticRegressionCV().fit(X_train[:, columns_supp], y_train)
    y_test_hat = clf.predict(X_test[:, columns_supp])
    y_train_hat = clf.predict(X_train[:, columns_supp])
    return [{
        "method": method,
        "train_accuracy": accuracy_score(y_train, y_train_hat),
        "train_f1": f1_score(y_train, y_train_hat),
        "test_accuracy": accuracy_score(y_test, y_test_hat),
        "test_f1": f1_score(y_test, y_test_hat),
        "n_features": columns_supp.sum()
    }]

def evaluate_methods(X, y, n_bins):
    results = []
    CMIM_max_features = 20
    for random_seed in range(5):
        X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.25, stratify=y, random_state=random_seed)
        binarizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform', subsample=None)
        variance_filter = VarianceThreshold(0.0)
        preprocess = make_pipeline(binarizer, variance_filter)
        Xt_train = preprocess.fit_transform(X_train)
        Xt_test = preprocess.transform(X_test)
        # JMI
        jmi_selector = JMISelector()
        jmi_selector.fit(Xt_train, y_train)
        results += base_model(Xt_train, Xt_test, y_train, y_test, jmi_selector.get_support().astype(bool), method="JMI")
        # CMIM
        cmin_selector = CMIMSelector(max_iter=CMIM_max_features)
        cmin_selector.fit(Xt_train, y_train)
        results += base_model(Xt_train, Xt_test, y_train, y_test, cmin_selector.get_support().astype(bool), method="CMIM")
        # LASSO
        lasso = LassoCV(cv=5, random_state=random_seed, selection='random', alphas=np.linspace(0.01, 0.05, 100), max_iter=10_000, tol=1e-4)
        lasso = lasso.fit(Xt_train, y_train)
        lasso_selector = SelectFromModel(lasso, prefit=True)
        results += base_model(Xt_train, Xt_test, y_train, y_test, lasso_selector.get_support().astype(bool), method="LASSO_binned")
        # SVM
        svm = LinearSVC(penalty='l1', dual=False, max_iter=100_000)
        grid = {'C': [10 ** i for i in range(-5, 5)]}
        svm_grid_search = GridSearchCV(svm, param_grid=grid, scoring='f1').fit(Xt_train, y_train)
        best_svm = svm_grid_search.best_estimator_.fit(Xt_train, y_train)
        svm_selector = SelectFromModel(best_svm)
        results += base_model(Xt_train, Xt_test, y_train, y_test, svm_selector.get_support().astype(bool), method='SVM_binned')

        # FastCMIM
        cmim_selected, scores = fast_cmim(Xt_train, y_train, n_selected_features=CMIM_max_features)
        cmim_mask = np.zeros(Xt_train.shape[1])
        cmim_mask[cmim_selected] = 1
        results += base_model(Xt_train, Xt_test, y_train, y_test, cmim_mask.astype(bool), method='SVM_binned')


        # Scaling
        scaler = StandardScaler()
        Xs_train = scaler.fit_transform(X_train)

        # LASSO not binned
        lasso = LassoCV(cv=5, random_state=random_seed, selection='random', alphas=np.linspace(0.01, 0.05, 100), max_iter=10_000, tol=1e-4)
        lasso = lasso.fit(Xs_train, y_train)
        lasso_selector = SelectFromModel(lasso, prefit=True)
        results += base_model(Xs_train, scaler.transform(X_test), y_train, y_test, lasso_selector.get_support().astype(bool), method="LASSO_unbinned")

        # SVM not binned
        svm = LinearSVC(penalty='l1', dual=False, max_iter=100_000)
        grid = {'C': [10 ** i for i in range(-5, 5)]}
        svm_grid_search = GridSearchCV(svm, param_grid=grid, scoring='f1').fit(Xs_train, y_train)
        best_svm = svm_grid_search.best_estimator_.fit(Xs_train, y_train)
        svm_selector = SelectFromModel(best_svm)
        results += base_model(Xs_train, scaler.transform(X_test), y_train, y_test, svm_selector.get_support().astype(bool), method='SVM_unbinned')
    return results

# Toxicity dataset:
[https://archive.ics.uci.edu/dataset/728/toxicity-2](https://archive.ics.uci.edu/dataset/728/toxicity-2)

In [3]:
# fetch dataset
toxicity = fetch_ucirepo(id=728)

# data (as pandas dataframes)
X = toxicity.data.features
y = toxicity.data.targets["Class"]

In [4]:
y = y.replace({
    "Toxic": 1,
    "NonToxic": 0})

C:\Users\Bartek\AppData\Local\Temp\ipykernel_15564\2588739897.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({


In [5]:
toxicity.variables['type'].value_counts()

type
Continuous     990
Integer        213
Categorical      1
Name: count, dtype: int64

In [6]:
results = evaluate_methods(X, y, 3)
toxicity_results = pd.DataFrame(results)
toxicity_results.to_csv("toxicity_results.csv")

E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 197 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 522 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 939 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 1121 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\prepr

# Period changer
[https://archive.ics.uci.edu/dataset/729/period+changer-2](https://archive.ics.uci.edu/dataset/729/period+changer-2)


In [15]:
period_changer = pd.read_csv("data/period_changer.csv")
X = period_changer.drop("Class", axis=1)
y = period_changer["Class"]

In [16]:
X.shape

(90, 1177)

In [17]:
y.value_counts()

Class
NoChanger    63
Changer      27
Name: count, dtype: int64

In [18]:
y = y.replace({
    "NoChanger": 0,
    "Changer": 1
})

C:\Users\Bartek\AppData\Local\Temp\ipykernel_15564\3081492086.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({


In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Columns: 1177 entries, MATS3v to MDEO-22
dtypes: float64(986), int64(191)
memory usage: 827.7 KB


In [20]:
results = evaluate_methods(X, y, 3)
period_changer_results = pd.DataFrame(results)
period_changer_results.to_csv("period_changer_results.csv")

E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 36 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 68 is constant and will be replaced with 0.
  warnin

# TUANDROMD
[https://archive.ics.uci.edu/dataset/855/tuandromd+(tezpur+university+android+malware+dataset)](https://archive.ics.uci.edu/dataset/855/tuandromd+(tezpur+university+android+malware+dataset))

In [3]:
tuandromd = pd.read_csv("data/TUANDROMD.csv")
X = tuandromd.drop("Label", axis=1)
y = tuandromd["Label"]

In [4]:
y.value_counts()

Label
malware     3565
goodware     899
Name: count, dtype: int64

In [5]:
y = y.replace({
    "malware": 1,
    "goodware": 0
})

C:\Users\Bartek\AppData\Local\Temp\ipykernel_19556\2315168950.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Columns: 241 entries, ACCESS_ALL_DOWNLOADS to Lorg/apache/http/impl/client/DefaultHttpClient;->execute
dtypes: int64(241)
memory usage: 8.2 MB


In [7]:
results = evaluate_methods(X, y, 2)
tuandromd_results = pd.DataFrame(results)
tuandromd_results.to_csv("tuandromd_results.csv")

E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 10 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 11 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:278: UserWarning: Feature 12 is constant and will be replaced with 0.
  warnings.warn(
E:\Studies\DataScience-3sem\MathematicalUnderpinningsOfMachineLearning\venv\lib\site-packages\sklearn\preprocessi